# MaterialsCoord benchmarking – sensitivity to perturbation analysis

This notebook demonstrates how to use MaterialsCoord to benchmark the sensitivity of bonding algorithms to structural perturbations. Perturbations are introduced according the Einstein crystal test rig, in which site is perturbed so that the distribution around the equilibrium position yields a normal distribution for each Cartesian component.

The perturbation complies thus with the expectation for an Einstein crystal,
in which the potential is given by $V(\delta r) = 0.5 k_\mathrm{spring} \delta r^2$, where
$k_\mathrm{spring}$ denotes the spring constant with which the sites are tethered to
their equilibrium position, and $\delta r$ is the distance of the site under
consideration from its equilibrium position.

The MaterialsCoord `Benchmark` class accepts a `perturb_sigma` option, which is equal to $(k_\mathrm{B}T/k_\mathrm{spring})^{0.5}$.

*Written using:*
- MaterialsCoord==0.1.0

*Authors: Hillary Pan, Alex Ganose (10/12/19)*

First, lets initialize the near neighbor methods we are interested in.

In [1]:
from pymatgen.analysis.local_env import BrunnerNN_reciprocal, EconNN, JmolNN, \
                                        MinimumDistanceNN, MinimumOKeeffeNN, MinimumVIRENN, \
                                        VoronoiNN, CrystalNN

nn_methods = [
    BrunnerNN_reciprocal(), EconNN(), JmolNN(), CrystalNN(), VoronoiNN(tol=0.5),
    MinimumDistanceNN(), MinimumOKeeffeNN(), MinimumVIRENN()
]

Next, import the benchmark and choose which structures we are interested in.

In [2]:
from materialscoord.core import Benchmark

structure_groups = ["common_binaries", "elemental", "A2BX4", "ABX3", "ABX4"]

Choose the initial and final perturbation sigma values to include, as well as the number of steps inbetween.

In [3]:
import numpy as np

initial_sigma = 0
final_sigma = 0.2
nsteps = 100

sigmas = np.linspace(initial_sigma, final_sigma, nsteps)

Run the benchmark with the perturbation turned on. Note we have disabled symmetry so that each perturbed site is treated separately.

In [ ]:
results = []
for sigma in sigmas:
    bm = Benchmark.from_structure_group(structure_groups, perturb_sigma=sigma, symprec=None)
    sigma_scores = bm.score(nn_methods, site_type="cation")
    results.append(sigma_scores.iloc[-1].values)

/Users/alexganose/dev/src/pymatgen/pymatgen/analysis/local_env.py:3532: UserWarning: CrystalNN: distance cutoffs set but no oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn("CrystalNN: distance cutoffs set but no oxidation "
/Users/alexganose/dev/src/pymatgen/pymatgen/analysis/local_env.py:3347: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  "CrystalNN: cannot locate an appropriate radius, "


Finally, plot the results.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os

plt_results = np.array(results).T

# define matplotlib style settings
style = {
    "font.sans-serif": ["Helvetica", "Arial"], "axes.labelsize": 16,
    "xtick.labelsize": 16, "ytick.labelsize": 16, "xtick.direction": "in",
    "ytick.direction": "in", "xtick.major.size": 8, "xtick.minor.size": 4,
    "ytick.major.size": 8, "ytick.minor.size": 4, "lines.linewidth": 2.5,
    "lines.markersize": 10, "axes.linewidth": 1.2, "xtick.major.width": 1.2,
    "xtick.minor.width": 1.2, "ytick.major.width": 1.2, "ytick.minor.width": 1.2,  
}

plt.style.use(style)
fig = plt.figure(figsize=(6, 6))
ax = plt.gca()

for method, y_vals in zip(nn_methods, plt_results):
    name = method.__class__.__name__
    ax.plot(sigmas, y_vals, label=name)

ax.set(ylabel="Benchmark score", xlabel="Sigma (Å)")

plt.legend()
plt.savefig(os.path.join("plots", "perturbation-tolerance.pdf"),
            dpi=400, bbox_inches="tight")
plt.show()